# Tutorial for setting up and simulating dsDNA with the NEAT-DNA model

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

sys.path.append('../../')

#The following two lines are the openABC imports relevant for NEAT-DNA simulations.
from openabc.forcefields.parsers import NEATDNAParser
from openabc.forcefields import MOFFNEATDNAModel

/private/tmp/openabc_pkgtest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this example we simulate a 2,000 bp dsDNA segment of the Lambda Phage DNA sequence with the  NEAT-DNA model. We provide a simple way to parse DNA topology and define all necessary interactions. Use class `NEATDNAParser` to parse each dsDNA. It is important to note that the parser requires the input structure be a complete double-stranded DNA structure. Either an atomistic structure may be passed and automatically coarse-grained, or an already coarse-grain structure may be utilized. As NEAT-DNA is capable of running long-sequences efficiently, we have added support for input mmCIF files in addition to PDB structures.

Note: If using mmCIF files generated with x3DNA's fiber module it is necessary to correct residue labels for proper parsing. This can quickly be done on Unix systems with the following command:

`cat fiber_DNA.cif|awk '{if($1=="ATOM"){$6="D"$6;$18=$6;};print $0}' > tmp.cif && mv tmp.cif fiber_DNA.cif`

Where `fiber_DNA.cif` is the x3DNA generated mmCIF file. In this example, we use an already corrected x3DNA generated structure.

The following line loads an atomistic mmCIF `fiber_LAMBDA_2000.cif`, coarse-grains it and parses it for simulation. The coarse-grained (CG) file is written to `cg_dsDNA.mmCIF`. Output CG file can be formatted as a PDB or mmCIF file, as specified by the `output_fmt` variable. In this case, the size of even the coarse-grained structure is too large for a PDB, so we must write to an mmCIF file instead

In [2]:
# parse dsDNA with atomistic model as input
dsDNA = NEATDNAParser.from_atomistic_mmCIF('input-structures/fiber_LAMBDA_2000.cif', 
                                           'cg_dsDNA.cif', output_fmt='mmCIF')

Processing mmCIF file: input-structures/fiber_LAMBDA_2000.cif
Reading: 100% (Done)


Now we can build the system and run simulation. We first build an instance of `MOFFNEATDNAModel`, then we append the dsDNA parser instance into it and set up the simulation. 

In [3]:
dna = MOFFNEATDNAModel()
dna.append_mol(dsDNA)
top = app.PDBxFile('cg_dsDNA.cif').getTopology()
dna.create_system(top, box_a=500, box_b=500, box_c=500)
salt_concentration = 100*unit.millimolar
temperature = 300*unit.kelvin
friction_coeff = 1.0 / unit.picosecond
timestep = 10 * unit.femtosecond
dna.add_dna_bonds(force_group=1)
dna.add_dna_angles(force_group=2)
dna.add_dna_breakable_fan_bonds(force_group=3)
dna.add_elec(salt_conc=salt_concentration, temperature=temperature, force_group=4)
dna.add_dna_bp_bonds(force_group=5)
dna.add_dna_bp_dihedrals(force_group=6)
dna.add_dna_bp_angles(force_group=7)
dna.save_system('dsDNA.xml')

platform_name = 'CPU'
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBxFile('cg_dsDNA.cif').getPositions()
dna.set_simulation(integrator, platform_name, init_coord=init_coord)
dna.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output.dcd'
dna.add_reporters(output_interval, output_dcd)
dna.simulation.context.setVelocitiesToTemperature(temperature)
dna.simulation.step(500)


Add DNA Bonds.
Add DNA Angles.
Add DNA Breakable Fan Bonds.
Add DNA electrostatic interactions with constant dielectric, no-switch, and 0.6 correction.
Add DNA Base-Pairing Bonds.
Add DNA Base-Pairing Dihedrals.
Add DNA Base-Pairing Angles.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,21804.502653806885,12242.54591188162,34047.04856568851,245.46800509138959,0
200,2.0000000000000013,23803.899172659258,14001.086638070974,37804.98581073023,280.72745905110276,7.96
300,2.99999999999998,24301.087401492245,14828.957928203856,39130.0453296961,297.3266137958792,7.97
400,3.9999999999999587,24913.791264375606,14792.018080331238,39705.809344706846,296.5859548813906,7.91
500,4.999999999999938,24945.08518951851,14722.592202775328,39667.67739229384,295.1939379113886,7.92
